In [1]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import graphviz 
# might need to install -- need *both* python and system
# so use Anaconda (or pip) to install graphviz, and also https://graphviz.org/download/
# this was a non-trivial install on Mac (depends on Xcode command-line tools)
# `pip install graphviz` and `brew install graphviz`
import datetime as dt

%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier, export_graphviz
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import confusion_matrix, mean_squared_error, mean_absolute_error

In [268]:
bb = pd.read_csv("../../Data/1999_2021/aggragate_daily_data_2000to2020.csv", index_col = "date")['energy_generated'].loc['2007-01-01':'2017-12-31'].to_frame()

weather = pd.read_csv("../../Data/historic_weather_all_SP_daily.csv", index_col = "date")

Changing columns to (station, measurement)

In [274]:
weather_tmp = weather.pivot(columns='station', values=['rain_mm', 'temp_max','temp_min'])



In [104]:
#really ugly. sorry.
weather_0 = weather_tmp.rename(columns={'rain_mm':'rain_mm_0', 'temp_max':'temp_max_0', 'temp_min':'temp_min_0'}).swaplevel(axis=1).sort_index(axis=1)
weather_7 = weather_tmp.shift(-7).rename(columns={'rain_mm':'rain_mm_7', 'temp_max':'temp_max_7', 'temp_min':'temp_min_7'}).swaplevel(axis=1).sort_index(axis=1)
weather_14 = weather_tmp.shift(-14).rename(columns={'rain_mm':'rain_mm_14', 'temp_max':'temp_max_14', 'temp_min':'temp_min_14'}).swaplevel(axis=1).sort_index(axis=1)
weather_30 = weather_tmp.shift(-30).rename(columns={'rain_mm':'rain_mm_30', 'temp_max':'temp_max_30', 'temp_min':'temp_min_30'}).swaplevel(axis=1).sort_index(axis=1)


In [212]:
rain = weather.pivot(columns='station', values=['rain_mm']).fillna(0)
rain_0 = rain.rename(columns={'rain_mm':'rain_mm_0'})
rain_7 = rain.shift(-7).rename(columns={'rain_mm':'rain_mm_7'})
rain_14 = rain.shift(-14).rename(columns={'rain_mm':'rain_mm_14'})
rain_30 = rain.shift(-30).rename(columns={'rain_mm':'rain_mm_30'})

rain_agg = bb.join(rain_0).join(rain_7).join(rain_14).join(rain_30)


/Users/jzhu/opt/anaconda3/lib/python3.8/site-packages/pandas/core/reshape/merge.py:643: UserWarning: merging between different levels can give an unintended result (1 levels on the left,2 on the right)
  warnings.warn(msg, UserWarning)


In [179]:
X_agg = rain_agg.drop('energy_generated', axis = 1)
y_agg = rain_agg.energy_generated
X_agg_train, X_agg_test, y_agg_train, y_agg_test = train_test_split(X_agg, y_agg, train_size = 0.8, random_state = 0)

In [213]:
rain_0 = rain_0.join(bb).dropna()
rain_7 = rain_7.join(bb).dropna()
rain_14 = rain_14.join(bb).dropna()
rain_30 = rain_30.join(bb).dropna()

/Users/jzhu/opt/anaconda3/lib/python3.8/site-packages/pandas/core/reshape/merge.py:643: UserWarning: merging between different levels can give an unintended result (2 levels on the left,1 on the right)
  warnings.warn(msg, UserWarning)
/Users/jzhu/opt/anaconda3/lib/python3.8/site-packages/pandas/core/reshape/merge.py:643: UserWarning: merging between different levels can give an unintended result (2 levels on the left,1 on the right)
  warnings.warn(msg, UserWarning)
/Users/jzhu/opt/anaconda3/lib/python3.8/site-packages/pandas/core/reshape/merge.py:643: UserWarning: merging between different levels can give an unintended result (2 levels on the left,1 on the right)
  warnings.warn(msg, UserWarning)
/Users/jzhu/opt/anaconda3/lib/python3.8/site-packages/pandas/core/reshape/merge.py:643: UserWarning: merging between different levels can give an unintended result (2 levels on the left,1 on the right)
  warnings.warn(msg, UserWarning)


In [292]:
# All stations, all lags predicting energy generated at B. Bonita

random_forest_rain_agg = RandomForestRegressor(max_features = 10, random_state = 1)

random_forest_rain_agg.fit(X_agg_train, y_agg_train)

pred_agg = random_forest_rain_agg.predict(X_agg_test)

Importance_agg = pd.DataFrame({'Importance':random_forest_rain_agg.feature_importances_*100}, 
                          index = X.columns)

Importance_agg = Importance_agg.sort_values(by = 'Importance', axis = 0)

print("random forest MSE:", mean_absolute_error(y_agg_test, pred_agg))

random forest MSE: 0.4692356022951492


In [246]:
Importance_agg = pd.DataFrame({'Importance':random_forest_rain_agg.feature_importances_*100}, 
                          index = X.columns)

Importance_agg = Importance_agg.sort_values(by = 'Importance', axis = 0)

In [247]:
Importance.sort_values('Importance',ascending=False).head(10)

,Importance
"(rain_mm_0, A729 VOTUPORANGA)",1.909062
"(rain_mm_0, A708 FRANCA)",1.754944
"(rain_mm_0, A706 CAMPOS DO JORDAO)",1.688148
"(rain_mm_0, A740 SAO LUIS DO PARAITINGA)",1.322364
"(rain_mm_7, A706 CAMPOS DO JORDAO)",1.287535
"(rain_mm_30, A712 IGUAPE)",1.275698
"(rain_mm_0, A746 BARRA DO TURVO)",1.264757
"(rain_mm_14, A712 IGUAPE)",1.252093
"(rain_mm_7, A729 VOTUPORANGA)",1.248769
"(rain_mm_30, A706 CAMPOS DO JORDAO)",1.232896


In [248]:
Importance.sort_values('Importance',ascending=False).tail(10)

,Importance
"(rain_mm_14, A744 BRAGANCA PAULISTA)",0.001361
"(rain_mm_7, A744 BRAGANCA PAULISTA)",0.000080
"(rain_mm_14, A770 SAO SIMAO)",0.000000
"(rain_mm_7, A770 SAO SIMAO)",0.000000
"(rain_mm_30, A770 SAO SIMAO)",0.000000
"(rain_mm_14, A771 SAO PAULO - INTERLAGOS)",0.000000
"(rain_mm_0, A771 SAO PAULO - INTERLAGOS)",0.000000
"(rain_mm_0, A770 SAO SIMAO)",0.000000
"(rain_mm_30, A771 SAO PAULO - INTERLAGOS)",0.000000
"(rain_mm_7, A771 SAO PAULO - INTERLAGOS)",0.000000


In [295]:
#Rain lagged 7 days predicting energy generated at B. Bonita
X_7 = rain_7.drop('energy_generated', axis = 1)
y_7 = rain_7.energy_generated
X_7_train, X_7_test, y_7_train, y_7_test = train_test_split(X_7, y_7, train_size = 0.8, random_state = 0)


random_forest_rain_7 = RandomForestRegressor(max_features = 42, random_state = 1)
random_forest_rain_7.fit(X_7_train, y_7_train)
pred_7 = random_forest_rain_7.predict(X_7_test)

Importance_7 = pd.DataFrame({'Importance':random_forest_rain_7.feature_importances_*100}, 
                          index = X_7.columns)

Importance_7 = Importance_7.sort_values(by = 'Importance', axis = 0)

print("random forest MSE:", mean_absolute_error(y_7_test, pred_7))

random forest MSE: 0.4957405273842981


In [253]:
Importance_7.sort_values('Importance', ascending=False).head(10)

,Importance
"(rain_mm_7, A712 IGUAPE)",6.453062
"(rain_mm_7, A706 CAMPOS DO JORDAO)",5.833458
"(rain_mm_7, A746 BARRA DO TURVO)",5.396074
"(rain_mm_7, A729 VOTUPORANGA)",4.995668
"(rain_mm_7, A740 SAO LUIS DO PARAITINGA)",4.475806
"(rain_mm_7, A708 FRANCA)",4.149706
"(rain_mm_7, A715 SAO MIGUEL ARCANJO)",4.085725
"(rain_mm_7, A755 BARUERI)",4.026940
"(rain_mm_7, A734 VALPARAISO)",3.936002
"(rain_mm_7, A701 SAO PAULO - MIRANTE)",3.113524


In [254]:
Importance_7.sort_values('Importance', ascending=False).tail(10)

,Importance
"(rain_mm_7, A766 REGISTRO)",0.494901
"(rain_mm_7, A762 DRACENA)",0.183845
"(rain_mm_7, A768 TUPA)",0.120371
"(rain_mm_7, A764 BEBDOURO)",0.109441
"(rain_mm_7, A767 SAO SEBASTIAO)",0.081960
"(rain_mm_7, A763 MARILIA)",0.070238
"(rain_mm_7, A769 CACHOEIRA PAULISTA)",0.054613
"(rain_mm_7, A744 BRAGANCA PAULISTA)",0.033478
"(rain_mm_7, A770 SAO SIMAO)",0.000000
"(rain_mm_7, A771 SAO PAULO - INTERLAGOS)",0.000000


In [294]:
#Rain lagged 14 days predicting energy generated at B. Bonita

X_14 = rain_14.drop('energy_generated', axis = 1)
y_14 = rain_14.energy_generated
X_14_train, X_14_test, y_14_train, y_14_test = train_test_split(X_14, y_14, train_size = 0.8, random_state = 0)

random_forest_rain_14 = RandomForestRegressor(max_features = 10, random_state = 1)

random_forest_rain_14.fit(X_14_train, y_14_train)

pred_14 = random_forest_rain_14.predict(X_14_test)

Importance_14 = pd.DataFrame({'Importance':random_forest_rain_14.feature_importances_*100}, 
                          index = X_14.columns)

Importance_14 = Importance_14.sort_values(by = 'Importance', axis = 0)

print("random forest MSE:", mean_absolute_error(y_14_test, pred_14))


random forest MSE: 0.48153677350932284


In [262]:
Importance_14.sort_values('Importance',ascending=False).head(10)

,Importance
"(rain_mm_14, A712 IGUAPE)",6.157848
"(rain_mm_14, A706 CAMPOS DO JORDAO)",5.568387
"(rain_mm_14, A746 BARRA DO TURVO)",5.333534
"(rain_mm_14, A708 FRANCA)",4.062979
"(rain_mm_14, A740 SAO LUIS DO PARAITINGA)",3.982146
"(rain_mm_14, A715 SAO MIGUEL ARCANJO)",3.857915
"(rain_mm_14, A701 SAO PAULO - MIRANTE)",3.620799
"(rain_mm_14, A729 VOTUPORANGA)",3.558242
"(rain_mm_14, A714 ITAPEVA)",3.428178
"(rain_mm_14, A718 RANCHARIA)",3.246155


In [263]:
Importance_14.sort_values('Importance',ascending=False).tail(10)

,Importance
"(rain_mm_14, A765 BERTIOGA)",0.435575
"(rain_mm_14, A769 CACHOEIRA PAULISTA)",0.269790
"(rain_mm_14, A762 DRACENA)",0.233060
"(rain_mm_14, A764 BEBDOURO)",0.172160
"(rain_mm_14, A767 SAO SEBASTIAO)",0.165994
"(rain_mm_14, A768 TUPA)",0.114605
"(rain_mm_14, A763 MARILIA)",0.112884
"(rain_mm_14, A744 BRAGANCA PAULISTA)",0.031205
"(rain_mm_14, A770 SAO SIMAO)",0.000000
"(rain_mm_14, A771 SAO PAULO - INTERLAGOS)",0.000000
